In [21]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from spotipy import oauth2
import csv
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import base64

In [22]:
def get_credentials():
    cid = '8c4cb3ef1c0c412181ea69d07b3df0ca'
    secret = 'a07e05c762a24e12933c4c118c6c18ca'
    redirect_uri = 'http://localhost:8910/callback'
    username = 'Arda Akça Büyük'
    return cid, secret, redirect_uri, username

In [23]:
def read_data(dataset_path):
    with open(dataset_path, 'r') as token_file:
        lines = csv.reader(token_file)
        dataset = list(lines)
        data = np.array(dataset)   
        return data

In [24]:
class KNNPredictor:
    def __init__(self, data):
        self.data = data
        self.preprocessing()
        self.train()
    
    def preprocessing(self):
        indices = []
        self.id_index = -1
        info = []
        choose = -1
        for i in range(self.data.shape[1]):
            if self.data[0][i] in ['key', 'artists', 'release_date', 'name', 'id']:
                indices.append(i)
                if self.data[0][i] == 'id':
                    self.id_index = i

            if self.data[0][i] in ['artists', 'name']:
                info.append(i)


        self.data = self.data[1:,:]
        np.random.shuffle(self.data)
        self.ids = np.array(self.data[:,self.id_index:self.id_index+1])
        song_info = np.array(self.data[:,info])
        self.data = np.delete(self.data, indices, axis=1)
        self.ids = np.ndarray.flatten(self.ids)
        self.data = self.data.astype('float64')
    
    def train(self):
        self.model = NearestNeighbors(n_neighbors = 26, algorithm = 'ball_tree')
        scalar = StandardScaler()
        scalar.fit(self.data)
        self.data = scalar.transform(self.data)
        self.model.fit(self.data)
    
    def predict(self, song_id):
        for j in range(self.data.shape[0]):
            if self.ids[j] == song_id:
                choose = j
        distances, indices = self.model.kneighbors([self.data[choose]])
        indices = np.ndarray.flatten(indices[:,1:])
        tracks = self.ids[indices]
        return tracks

In [34]:
def create_playlist(playlist_name, tracks, username, client_id, client_secret, redirect_uri):
    scope = "playlist-modify-public"
    token = util.prompt_for_user_token(username,scope,client_id=client_id,client_secret=client_secret,redirect_uri=redirect_uri) 
    sp = spotipy.Spotify(auth=token)
    sp.user_playlist_create(username, name=playlist_name)
    playlists = sp.user_playlists(username)
    tracks = np.ndarray.flatten(tracks)
    tracks = list(tracks)
    sp.user_playlist_add_tracks(username, playlists['items'][0]['id'], tracks)

In [35]:
def main():
    cid, secret, redirect_uri, username = get_credentials()
    data = read_data('./data/data.csv')
    predictor = KNNPredictor(data)
    song_id = input('Enter song ID: ')
    tracks = predictor.predict(song_id)
    username = input('Enter username: ')
    playlist_name = input('Enter playlist name: ')
    create_playlist(playlist_name, tracks, username, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

In [36]:
main()

Enter song ID: 7wBJfHzpfI3032CSD7CE2m
Enter username: ardabyk07
Enter playlist name: STARGAZING (kNN)
